# Data Wrangling mit Python I

## Einführung

Die Programmiersprache Python hat in den letzten Jahren unglaublich an Beliebtheit gewonnen. Besondes im Bereich Maschinelles Lernen und

In [ ]:
In diesem Jupyter-Notebook werden grundlegende 

## using the Jupyter-Notebook:

### [Keyboard Shortcuts](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Custom%20Keyboard%20Shortcuts.html)

| Command | Action |
| --- | --- |
|Ctrl + Enter | run selected cells |
|Alt + Enter | run the current cell, insert below |
|Shift + Enter | run the current cell, select below |
|Ctrl + S | save and checkpoint |

### [Magic-Commands and othes](https://ipython.readthedocs.io/en/stable/interactive/magics.html)

| Command | Action |
| --- | --- |
| %load python_file.py| load code from python_file.py into Cell |
| %run python_file.py| run python_file.py |
| %time | times how long a cell needs to finish |
| %ls | shows file in the current directory |
| %pwd | shows path of the current directory |

Want more? Here are [28 Tips, Tricks and Shortcuts](https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/).

In [ ]:
# import all the packages we need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Show visualisation inside the notebook (not needed anymore in newer versions)
%matplotlib inline 

In [ ]:
%pwd

### Working with the price dataset

In [ ]:
# create Dataframe from CSV
# use utf-8
df_price = pd.read_csv("./FlourishOA_Data/api_price11-13-17.csv",  encoding="ISO-8859-1", sep=',', decimal='.')
# we can read json, excel, sql, and more ... https://pandas.pydata.org/pandas-docs/stable/reference/io.html

In [ ]:
# view first lines of dataset
df_price.head()
# df.tail() for last lines and df for (reduced) view of the whole Dataframe 

In [ ]:
# get basic information about datatypes, entries and columns
df_price.info()
# more infos about datatypes in pandas: https://pbpython.com/pandas_dtypes.html

In [ ]:
# lets change this!
df_price["date_stamp"] = pd.to_datetime(df_price['date_stamp'])

In [ ]:
# lets have a look again
df_price.info()

In [ ]:
# is column `id` really a unique identifier?
unique_id = df_price["id"].nunique()
count_id = df_price["id"].count()
ununique_count = count_id - unique_id
ununique_count

### Working with the journal dataset

In [ ]:
# create Dataframe from CSV
df_journal = pd.read_csv("./FlourishOA_Data/api_journal11-13-17.csv",  encoding="ISO-8859-1", sep=',', decimal='.')
# we can read json, excel, sql, and more ... https://pandas.pydata.org/pandas-docs/stable/reference/io.html

In [ ]:
# view first 10 lines of dataset
df_journal.head(10)

In [ ]:
# get basic information about datatypes, entries and columns
df_journal.info()

In [ ]:
# are there only 0 and 1 in `is_hybrid`?
df_journal.groupby("is_hybrid").count()

In [ ]:
# change data-type for is_hybrid
df_journal["is_hybrid"] = df_journal["is_hybrid"].astype(bool)

In [ ]:
# lets have a look again
df_journal.info()

In [ ]:
# look at the different categories
df_journal.groupby("category", sort=True).count()

In [ ]:
# change all categorys to uppercase
category_series = df_journal["category"]
df_journal["category"] = df_journal["category"].map(lambda x: x if x is np.nan else x.upper() )

In [ ]:
# look at the different categories again
df_journal.groupby("category", sort=True).count()


In [ ]:
# lets have a look at one specific category "AGRICULTURE" 
category_series_filter = df_journal[df_journal["category"].notnull()]["category"].map(lambda x: "AGRICULTURE" in x)
category_series_filter
df_journal[df_journal["category"].notnull()][category_series_filter]
df_journal[df_journal["category"].notnull()][category_series_filter].groupby("category").count()

In [ ]:
# different seperator in mulitcategories. Lets find them
category_series_filter = df_journal[df_journal["category"].notnull()]["category"].map(lambda x: "." in x)
df_journal[df_journal["category"].notnull()][category_series_filter]
df_journal[df_journal["category"].notnull()][category_series_filter].groupby("category").count()

In [ ]:
# replace "." with " |"
df_journal["category"] = df_journal["category"].map(lambda x: x if x is np.nan else x.replace(".", " |"))


In [ ]:
# look at Results with specific category "PHILOSOPHY"
category_series_filter = df_journal[df_journal["category"].notnull()]["category"].str.contains("PHILOSOPHY") #df[df["category"].notnull()]["category"].str.contains()

df_journal[df_journal["category"].notnull()][category_series_filter]
df_journal[df_journal["category"].notnull()][category_series_filter].groupby("category").count()


In [ ]:
# lets get a list of all Categories

# 1. split categories
def category_string_to_list(category_string):
    category_list = [x.strip() for x in str(category_string).split('|')]
    return category_list

category_list_series = df_journal["category"].map(category_string_to_list)
category_list_series

In [ ]:
# 2. creating a set of all categorys ant get it length
category_set = set()

category_lists = category_list_series.tolist()
for category_list in category_lists:
    category_set.update(category_list)

print(category_set)
len(category_set)-1

### Merge journal and price Dataframes

In [ ]:
df_journal.head()

In [ ]:
df_price.head()

In [ ]:
# get Number of ununique columns for possible merge
def get_ununique_count(df, column):
    return df[column].count() - df[column].nunique()

print(df_price.columns)
for column in df_price.columns:
    print(column, get_ununique_count(df_price, column))

print(df_journal.columns)
for column in df_journal.columns:
    print(column)
    print(get_ununique_count(df_journal, column))


In [ ]:
# merging two dataframes

df_merge = pd.merge(df_journal, df_price, how='inner', left_on='issn', right_on='journal_id',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)[["journal_name", "price", "category", "influence_id"]]

In [ ]:
# lets have a look at the merged data
print(df_merge.info())
df_merge.head()

In [ ]:
# get generall informations about the price of articles
df_merge.price.describe()

In [ ]:
# creat a lineplot
df_merge.price.plot()

In [ ]:
# create a histogramm with 16 bins
df_merge.price.plot.hist(bins=16)

In [ ]:
# get all articles with price 0
df_merge[df_merge.price == 0].count()

In [ ]:
# get grouped information about journals
table = pd.pivot_table(df_merge, values=["price"], index="journal_name", aggfunc={'price': [min, max ,np.sum, len, np.mean]})
table.sort_values(by=("price", "sum"), ascending=False)

In [ ]:
# export table to excel-format
table.to_excel("././FlourishOA_Data/journal-price_pivot-table.xlsx")
# want more possibilities to work with excel? https://pbpython.com/improve-pandas-excel-output.html